In [ ]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np

model = load_model("emotion_model.h5")
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
cap = cv2.VideoCapture(0)

last_face_position = None
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret: break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    display_frame = frame.copy()

    # Hanya deteksi ulang setiap 5 frame untuk stabilitas
    if frame_count % 5 == 0 or last_face_position is None:
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=6)
        if len(faces) > 0:
            # Ambil wajah terbesar (terdekat ke kamera)
            faces = sorted(faces, key=lambda f: f[2] * f[3], reverse=True)
            last_face_position = faces[0]

    frame_count += 1

    if last_face_position is not None:
        (x, y, w, h) = last_face_position
        roi = gray[y:y+h, x:x+w]
        try:
            roi = cv2.resize(roi, (48, 48))
            roi = roi.astype('float32') / 255.0
            roi = np.expand_dims(roi, axis=0)
            roi = np.expand_dims(roi, axis=-1)

            preds = model.predict(roi, verbose=0)[0]
            label = emotion_labels[np.argmax(preds)]
            confidence = np.max(preds)

            cv2.rectangle(display_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(display_frame, f"{label} ({confidence:.2f})", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        except:
            pass

    cv2.imshow("Ekspresi Wajah", display_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
